In [ ]:
# Update viresclient 
!pip install git+https://github.com/ESA-VirES/VirES-Python-Client.git@staging
# Update xarray to get the new html repr
!pip install -U xarray

### Imports

In [ ]:
from viresclient import SwarmRequest
import matplotlib.pyplot as plt
# import holoviews as hv
# import hvplot.xarray
import panel as pn
pn.extension()

### VirES access

In [ ]:
# server_url = 'https://staging.viresdisc.vires.services/ows'
server_url = 'https://vires.services/ows'

def fetch_data(
            observatory="ABG",
            models=None,
            start_time='2016-01-01T00:00:00Z',
            end_time='2016-01-10T00:00:00Z',
            cadence="M"
        ):
    request = SwarmRequest(server_url)
    request.set_collection(
       f'SW_OPER_AUX_OBS{cadence}2_:{observatory}',
        verbose=False
    )
    request.set_products(
        measurements=['B_NEC', 'F', 'IAGA_code', 'Quality'],
        models=models,
        residuals=True if models else False
    )
    data = request.get_between(
        start_time, end_time,
        asynchronous=True,
        show_progress=False)
    return data

### Dashboard setup

In [ ]:
models = [
    None,
    '"CHAOS-Core"',
    '"CHAOS-Core" + "CHAOS-Static"',
    '"CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA"',
    '"CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA" + "MIO_SHA_2C"'
]

panes = {
    "html": pn.pane.HTML(),
    "mpl": pn.pane.Matplotlib(),
#     "hvp": pn.pane.HoloViews()
}
widgets = {
    "select_obs": pn.widgets.Select(name='Select observatory:', options=['LER', 'ESK', 'HAD']),
    "input_start": pn.widgets.TextInput(name='Start time:', value='2016-01-01T00:00:00Z'),
    "input_end": pn.widgets.TextInput(name='End time:', value='2016-01-02T00:00:00Z'),
    "select_model": pn.widgets.Select(name='Remove Model:', options=models),
    "select_resampling": pn.widgets.Select(name='Resampling:', options=[None, '1H', '1D']),
    "button": pn.widgets.Button(name='Click to generate data', button_type='primary'),
#     "download": pn.widgets.FileDownload()
}

def button_outcome(event):
    widgets["button"].name = "Loading..."
    # Fetch dataset
    selected_model = widgets["select_model"].value
    data = fetch_data(
        observatory=widgets["select_obs"].value,
        start_time=widgets["input_start"].value,
        end_time=widgets["input_end"].value,
        models=[f"Model = {selected_model}"]
            if selected_model else None
    )
    ds = data.as_xarray()
    # Display dataset html
    panes["html"].object = ds._repr_html_()
    # Display figure
    plotvar = 'B_NEC_res_Model' if selected_model else 'B_NEC'
    selected_resampling = widgets["select_resampling"].value
    fig, axes = plt.subplots(ncols=3, figsize=(15, 3))
    for i, NEC in enumerate("NEC"):
        ds[plotvar].sel(NEC=NEC).plot.line(x="Timestamp", ax=axes[i])
        if selected_resampling:
            (ds[plotvar].sel(NEC=NEC)
                .resample(Timestamp=selected_resampling).mean()
                .plot.line(x="Timestamp", ax=axes[i])
            )
        axes[i].set_ylabel(None)
        axes[i].grid()
    axes[0].set_ylabel(f"Observatory: {widgets['select_obs'].value}\n\n{plotvar} [nT]")
    panes["mpl"].object = fig
#     panes["hvp"].object = ds.hvplot.line(
#         x="Timestamp", y=plotvar, col="NEC",
#         shared_axes=False, grid=True, frame_width=350, frame_height=200
#     )
    # Reset button
    widgets["button"].name = 'Click to generate data'

widgets["button"].on_click(button_outcome)

dashboard = pn.Column(
    pn.Row(*[widgets[i] for i in ["select_obs", "input_start", "input_end"]],  width=800),
    pn.Row(*[widgets[i] for i in ["select_model", "select_resampling"]], width=800),
    pn.Row(widgets["button"], width=800),
    panes["mpl"],
#     panes["hvp"],
    panes["html"],
)

### Display the dashboard

This dashboard fetches and displays the minute data from one of the three UK magnetic observatories ([see the usage terms](ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/minute/README)). You can optionally choose to remove a geomagnetic model prediction - this will be evaluated on demand, so can be quite slow with the magnetospheric (MMA) and ionospheric (MIO) models. You can additionally resample the output (using pandas [resampling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)) - to hourly (`1H`) or daily rate (`1D`).

We show the NEC (North, East, Centre) components of the field in geocentric spherical coordinates.

Of course a lot more is possible with this approach, and this is only meant to serve as a demonstration!  
(it uses the [Panel](https://panel.holoviz.org/index.html) dashboarding tool)

In [ ]:
dashboard